<a href="https://colab.research.google.com/github/AseelAbuAl-Rub/NLP-Project/blob/main/Task_1%2C2_and_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this code, we load the data from google drive and stored it in data frames for each file then concat train negative and positive in one data frame name: train data, also concat test in one data frame name: test data frame; convert the data type of labelling to boolean. We apply a cleaning process: remove numbers, special characters, and punctuation marks then tokenized text, removed stop words and apply Lemmatization.
As a vectorizing representation text, we used TF-IDF on 'cleaned_text' training data, then fit the test data on the trained and transformed it. Two classifiers used SVM and logistic regression just to compare the accuracy of there predictions.
Logistic regression accuracy is 0.703 where SVM 0.698

In [ ]:
import os
import pandas as pd
import nltk
import re
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#@title Authenticate & authorize with google drive
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#@title Sets up a local directory 

local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

#    https://developers.google.com/drive/v2/web/search-parameters
train_neg_id = '1PXqBtQRUb4gdoSbzz-kUEXq7F9O6wTPz'
train_pos_id = '1i_m3qQC3OPAUckbljbiZ-6HfTQHPrNyh'

test_neg_id = '1qQD54k2x42mbzAKRrQkL4GhdtRgGVRAK'
test_pos_id = '1yTaxzpANeSaSk7vwCIEByRJmzEqhoIcz'

local_download_path = os.path.expanduser('~/data')

In [ ]:
#@title Read From Drive & Store In Dataframe Function
def read_from_drive(drive, parent_folder_id, local_download_path):
    df = pd.DataFrame()  # Instantiate an empty DataFrame
    file_list = drive.ListFile({'q': f"'{parent_folder_id}' in parents"}).GetList()

    for f in file_list:
        fname = os.path.join(local_download_path, f['title'])
        f_ = drive.CreateFile({'id': f['id']})
        f_.GetContentFile(fname)
        
        with open(fname, 'r') as file:
            file_content = file.read()
            df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
    
    return df

In [ ]:
#@title Apply (read from drive and store in df) Function
train_neg = read_from_drive(drive, train_neg_id, local_download_path)
train_neg['flag'] = 0
train_pos = read_from_drive(drive, train_pos_id, local_download_path)
train_pos['flag'] = 1

test_neg = read_from_drive(drive, test_neg_id, local_download_path)
test_neg['flag'] = 0
test_pos = read_from_drive(drive, test_pos_id, local_download_path)
test_pos['flag'] = 1


<ipython-input-4-6a27e60d113d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-4-6a27e60d113d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-4-6a27e60d113d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-4-6a27e60d113d>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Tweets': [file_content]}), ignore_index=True)
<ipython-input-4

In [ ]:
#@title Concat the data
train_data = pd.concat([train_neg, train_pos], ignore_index=True)
test_data = pd.concat([test_neg, test_pos], ignore_index=True)

In [ ]:
#@title Load stop words
#Stop Words
stop_words=set(stopwords.words('english'),)

In [ ]:
#@title Remove Characters And Number Function 
def remove_num_special_char(text):
    cleaned_text = re.sub(r"[^a-zA-Z\s]", "", text)
    return cleaned_text



In [ ]:
#@title Remove Stop Words & Lemmatization Function
def remove_stopwords_lem(text):
    lemmatizer = WordNetLemmatizer()
    filtered_tokens = [lemmatizer.lemmatize(token) for token in text if token not in stop_words]
    return filtered_tokens
    

In [ ]:
#@title Apply (remove character and num) function
train_data['cleaned_text'] = train_data['Tweets'].apply(remove_num_special_char)

test_data['cleaned_text'] = test_data['Tweets'].apply(remove_num_special_char)

In [ ]:
#@title Tokenize text
train_data['tokenized_text'] = train_data['cleaned_text'].apply(lambda x: word_tokenize(x))

test_data['tokenized_text'] = test_data['cleaned_text'].apply(lambda x: word_tokenize(x))

In [ ]:
#@title Convert to lower case 
train_data['tokenized_text'] = train_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

test_data['tokenized_text'] = test_data['tokenized_text'].apply(lambda tokens: [token.lower() for token in tokens])

In [ ]:
#@title Apply (remove stop words & lemmatizatio) function
train_data['cleaned_tokens'] = train_data['tokenized_text'].apply(remove_stopwords_lem)

test_data['cleaned_tokens'] = test_data['tokenized_text'].apply(remove_stopwords_lem)

In [ ]:
#@title Final version of cleaned text
train_data['cleaned_text']=train_data['cleaned_tokens'].apply(lambda x: ' '.join(x))

test_data['cleaned_text']=test_data['cleaned_tokens'].apply(lambda x: ' '.join(x))

In [ ]:
#@title TF-IDF
v_train_data = TfidfVectorizer()
x_train_data = v_train_data.fit_transform(train_data['cleaned_text'])
fnames_train_data=v_train_data.get_feature_names_out()
train_data_tfidf= pd.DataFrame(x_train_data.todense().tolist(), columns= fnames_train_data)


x_test_data = v_train_data.transform(test_data['cleaned_text'])
fnames_test_data=v_train_data.get_feature_names_out()
test_data_tfidf= pd.DataFrame(x_test_data.todense().tolist(), columns= fnames_test_data)




In [ ]:
test_data_tfidf

,aames,abandon,abdicating,ability,able,ably,aboutbr,abraham,abre,abrupt,...,zhang,zhuangzhuang,zimmerframe,zip,zoe,zombie,zombiez,zone,zoom,zslasher
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.078732,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
195,0.0,0.0,0.0,0.0,0.063456,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.091226,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#@title Logistic Regression

classifier = LogisticRegression()
classifier.fit(train_data_tfidf, train_data['flag'])
predictions = classifier.predict(test_data_tfidf)

# Evaluate the accuracy
accuracy = accuracy_score(test_data['flag'], predictions)
accuracy

0.7035175879396985

In [ ]:
#@title Support Vector Machine (SVM)

classifier = SVC()
classifier.fit(train_data_tfidf, train_data['flag'])
predictions = classifier.predict(test_data_tfidf)

# Evaluate the accuracy
accuracy = accuracy_score(test_data['flag'], predictions)
accuracy

0.6984924623115578